In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
import glob
import datetime
from sklearn.preprocessing import robust_scale
from tqdm.auto import tqdm

from preprocessing_utils import *
from AdvASLTM import *

In [3]:
T = 15

date_limit_train_validation = datetime.datetime(year = 2016, month=5, day=1)
features = {
    "preprocessed_open": lambda df : df["Open"] / df["Close"] - 1,
    "preprocessed_high": lambda df : df["High"] / df["Close"] - 1,
    "preprocessed_low": lambda df : df["Low"] / df["Close"] - 1,
    "preprocessed_close": lambda df : df["Close"].pct_change(),
    "preprocessed_adj_close": lambda df : df["Adj Close"].pct_change(),
    "preprocessed_5-day": lambda df : (df["Adj Close"].rolling(5).mean() / df["Adj Close"]) -1,
    "preprocessed_10-day": lambda df : (df["Adj Close"].rolling(10).mean() / df["Adj Close"]) -1,
    "preprocessed_15-day": lambda df : (df["Adj Close"].rolling(15).mean() / df["Adj Close"]) -1,
    "preprocessed_20-day": lambda df : (df["Adj Close"].rolling(20).mean() / df["Adj Close"]) -1,
    "preprocessed_25-day": lambda df : (df["Adj Close"].rolling(25).mean() / df["Adj Close"]) -1,
    "preprocessed_30-day": lambda df : (df["Adj Close"].rolling(30).mean() / df["Adj Close"]) -1,
    "preprocessed_volume": lambda df : df["Volume"]
}



# Data : Gather & Preprocess

In [7]:
raw_data_path = "data/stocknet-dataset/price/raw/*.csv"
raw_data_pathes = glob.glob(raw_data_path)

X_train, y_train = None, None
X_validation, y_validation = None, None
#Iterate through each stock RAW data
for path in tqdm(raw_data_pathes): 
    stock_df = pd.read_csv(path, parse_dates=["Date"],index_col="Date")
    
    ## Generate labels
    labelling(stock_df)

    ## Preprocessing
    stock_df.sort_index(inplace= True)
    stock_df.dropna(inplace = True)

    ### Apply features functions
    for feature_key in features.keys(): 
        stock_df[feature_key] = robust_scale(features[feature_key](stock_df))
    stock_df.dropna(inplace = True)

    train_stock_df = stock_df[stock_df.index < date_limit_train_validation]
    validation_stock_df = stock_df[stock_df.index >= date_limit_train_validation]

    ## Generate sequences
    X_stock_train, y_stock_train = generate_sequences(df = train_stock_df, features_columns= features.keys(), T = T )
    X_stock_validation, y_stock_validation = generate_sequences(df = validation_stock_df, features_columns= features.keys(), T = T)

    if X_train is None: X_train = X_stock_train
    else : X_train = np.concatenate([X_train, X_stock_train], axis = 0)
    if X_validation is None: X_validation = X_stock_validation
    else : X_validation = np.concatenate([X_validation, X_stock_validation], axis = 0)
    if y_train is None: y_train = y_stock_train
    else : y_train = np.concatenate([y_train, y_stock_train], axis = 0)
    if y_validation is None: y_validation = y_stock_validation
    else : y_validation = np.concatenate([y_validation, y_stock_validation], axis = 0)

# Shuffle X and y
X_train, y_train = shuffled_X_y(X_train, y_train)
X_validation, y_validation = shuffled_X_y(X_validation, y_validation)
X_train.shape, y_train.shape, X_validation.shape, y_validation.shape

  0%|          | 0/87 [00:00<?, ?it/s]

((44268, 15, 12), (44268,), (14237, 15, 12), (14237,))

# Tuner search

In [17]:
model = AdvALSTM(
    units = 16, 
    epsilon = 0.001, 
    beta = 0.05, 
    learning_rate = 1E-2, 
    l2 = 0.001, 
    attention = True, 
    hinge = True,
    adversarial_training = True
)
model.fit(
    X_train, y=  y_train,
    validation_data = (X_validation, y_validation),
    epochs = 300,
    batch_size = 1024
)

Epoch 1/300


2023-03-22 18:10:28.990475: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] function_optimizer failed: INVALID_ARGUMENT: Input 0 of node Adam/gradients/zeros_like_4 was passed float from sequential_22/lstm_11/PartitionedCall:7 incompatible with expected variant.
2023-03-22 18:10:29.010923: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] shape_optimizer failed: OUT_OF_RANGE: src_output = 31, but num_outputs is only 30
2023-03-22 18:10:29.021932: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] layout failed: OUT_OF_RANGE: src_output = 31, but num_outputs is only 30
2023-03-22 18:10:29.049813: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:903] tfg_optimizer{} failed: INVALID_ARGUMENT: Input 0 of node Adam/gradients/zeros_like_4 was passed float from sequential_22/lstm_11/PartitionedCall:7 incompatible with expected variant.
	when importing GraphDef to MLIR module in GrapplerHook
2023-03-22 18:10:29.057674: E tensorflow/core/grappler/optimiz

44/44 [==============================] - 6s 46ms/step - loss: nan - hinge_acc: 0.4872 - val_loss: nan - val_hinge_acc: 0.4631
Epoch 2/300
44/44 [==============================] - 2s 35ms/step - loss: nan - hinge_acc: 0.4850 - val_loss: nan - val_hinge_acc: 0.4631
Epoch 3/300
44/44 [==============================] - 1s 33ms/step - loss: nan - hinge_acc: 0.4850 - val_loss: nan - val_hinge_acc: 0.4631
Epoch 4/300
44/44 [==============================] - 1s 32ms/step - loss: nan - hinge_acc: 0.4850 - val_loss: nan - val_hinge_acc: 0.4631
Epoch 5/300
44/44 [==============================] - 1s 34ms/step - loss: nan - hinge_acc: 0.4850 - val_loss: nan - val_hinge_acc: 0.4631
Epoch 6/300
44/44 [==============================] - 1s 34ms/step - loss: nan - hinge_acc: 0.4850 - val_loss: nan - val_hinge_acc: 0.4631
Epoch 7/300
44/44 [==============================] - 2s 39ms/step - loss: nan - hinge_acc: 0.4850 - val_loss: nan - val_hinge_acc: 0.4631
Epoch 8/300
44/44 [===========================